In [1]:
import numpy as np

def get_augmented_train(model, x_train, y_train, x_test, sample_rate=0.2):
    num_sample = int(len(x_test)* sample_rate)

    model.fit(x_train,y_train,batch_size=128)
    psudo_lables = model.predict(x_test)

    # sort by cross_entropy
    devetorized_psudo_lables = np.max(psudo_lables,axis=0)
    idx = devetorized_psudo_lables.argsort()
    temp_x_test = x_test[idx]
    psudo_lables = psudo_lables[idx]
    # concatenate
    x_train = np.concatenate((x_train, temp_x_test[:num_sample]), axis=0)
    y_train = np.concatenate((y_train, psudo_lables[:num_sample]), axis=0)
    # # delete train_data from x_test
    # x_test = temp_x_test[num_sample:]
    # shuffle x_train
    idx = list(range(len(x_train)))
    np.random.shuffle(idx)
    x_train = x_train[idx]
    y_train = y_train[idx]
    return x_train, y_train

In [2]:
# data preprocess
from utils import to_one_hot
from keras.datasets import cifar10

(x_test_original, y_test_original), (x_train_original, y_train_original) = cifar10.load_data()
y_train_vetorized = to_one_hot(y_train_original, dimension=10)
y_test_vetorized = to_one_hot(y_test_original, dimension=10)

In [3]:
print(x_train_original.shape)
print(y_train_original.shape)
print(y_train_original)

(10000, 32, 32, 3)
(10000, 1)
[[3]
 [8]
 [8]
 ...
 [5]
 [1]
 [7]]


In [4]:
# get 400 different labels of data
def getDataset(x_train_original, y_train_originnal, each_size=400):
    print(y_train_originnal[:10])
    x_train = x_train_original[np.argmax(y_train_originnal,axis=1)==0]
    x_train = x_train[:each_size]
    y_train = [0] * each_size
    y_train = np.array(y_train).astype('int64')
    for i in range(1,10):
        appending_x_train = x_train_original[np.argmax(y_train_originnal,axis=1)==i]
        x_train = np.concatenate((x_train, appending_x_train[:each_size]),axis=0)
        appending_y_train = np.array([i] * each_size).astype('int64')
        y_train = np.concatenate((y_train, appending_y_train[:each_size]),axis=0)

    return x_train,to_one_hot(y_train, dimension=10)


In [5]:
x_train, y_train = getDataset(x_train_original,y_train_vetorized)
print(len(x_train))
print(len(y_train))

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
4000
4000


In [6]:
from keras import models, layers

def getModel():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(64,(3,3), activation='relu'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(128,(3,3),activation='relu'))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128,activation='relu'))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.summary()
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [7]:
def my_fit(model, x_train, y_train, x_test, epoch_nums=50):
    for i in range(epoch_nums):
        x_train, y_train= get_augmented_train(model, x_train,y_train,x_test,0.2)
        if i%5==0 :
            print('length of x_train:{}, y_train:{}, x_test:{}'.format(len(x_train),len(y_train),len(x_test)))

In [8]:
print(len(x_test_original))
model = getModel()
my_fit(model, x_train,y_train,x_test_original)

50000


2022-12-18 14:33:50.219384: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-18 14:33:50.234549: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-18 14:33:50.235061: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-18 14:33:50.237271: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0

2022-12-18 14:33:51.549852: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8100
2022-12-18 14:33:51.850493: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


32/32 [==============================] - 2s 7ms/step - loss: 6.3991 - accuracy: 0.1252
length of x_train:4010, y_train:4010, x_test:50000
32/32 [==============================] - 0s 7ms/step - loss: 1.9964 - accuracy: 0.2795
length of x_train:4060, y_train:4060, x_test:50000
33/33 [==============================] - 0s 6ms/step - loss: 1.6562 - a

In [9]:
from sklearn.metrics import accuracy_score

predictions = np.argmax(model.predict(x_test_original),axis=1)
print('the size of test dataset is {}'.format(len(predictions)))
print('accuracy is {}'.format(accuracy_score(predictions ,y_test_original)))

the size of test dataset is 50000
accuracy is 0.47816


In [10]:
# configure tensorboard
%load_ext tensorboard
%tensorboard --logdir logs

Launching TensorBoard...

In [11]:
import tensorflow as tf
tf_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

2022-12-18 14:35:12.226780: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-12-18 14:35:12.226799: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-12-18 14:35:12.226842: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1614] Profiler found 1 GPUs
2022-12-18 14:35:12.226990: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory
2022-12-18 14:35:12.227079: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory
2022-12-18 14:35:12.227108: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1666] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be 

In [12]:
anotherModel = getModel()
anotherModel.fit(x_train, y_train, epochs=50, callbacks=[tf_callback])
# using browser, go to localhost:6006 and see what have been created tensorboard




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)              

2022-12-18 14:35:12.893026: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-12-18 14:35:12.893045: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-12-18 14:35:12.893064: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1666] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2022-12-18 14:35:12.897479: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-12-18 14:35:12.897530: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1757] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2022-12-18 14:35:12.898372: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2022-12-18 14:35:12.898729: I tensorflow

125/125 [==============================] - 1s 3ms/step - loss: 2.8073 - accuracy: 0.1605
Epoch 2/50
125/125 [==============================] - 0s 2ms/step - loss: 2.0402 - accuracy: 0.2492
Epoch 3/50
125/125 [==============================] - 0s 2ms/step - loss: 1.8730 - accuracy: 0.3055